In [1]:
import sys
from pathlib import Path

# Specify the path to your modules directory
module_path = Path("/Users/elyestabbane/Documents/UNICOG/2-Experiments/replayseq/1-Scripts/analysis_scripts")

# Add the directory to sys.path if it's not already included
if module_path not in sys.path:
    sys.path.append(str(module_path))

# Import your modules
from modules import *
import pandas as pd
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [2]:
import os
import mne
from mne_bids import BIDSPath, write_raw_bids, write_meg_calibration, write_meg_crosstalk
import json
import numpy as np

# Path to BIDS directory
path_bids = "/Volumes/T5_EVO/1-experiments/REPLAYSEQ/2-Data/BIDS"

In [3]:
subject=16

In [4]:
with open(path_json_file, 'r') as file:
        bad_channels_dict = json.load(file)

In [5]:
lab=bad_channels_dict[f"sub-{subject:02}"]['lab']
if lab == 'icm':
    original_data_path = os.path.join(path_root, 'Data_ICM', f"sub-{subject:02}")
else:
    original_data_path = os.path.join(path_root, 'Data_neurospin', f"sub-{subject:02}")

In [6]:
run=2
run_str=f'run{run:02}'

if lab == 'icm':
    data_path = os.path.join(original_data_path, f'{run_str}.fif')
else:
    data_path = os.path.join(original_data_path, f'{run_str}_raw.fif')

if not os.path.exists(data_path):
    print(f"File not found: {data_path}, skipping run {run_str}")

In [7]:
raw = mne.io.read_raw_fif(data_path, allow_maxshield=True, preload=True)

Opening raw data file /Volumes/T5_EVO/1-experiments/REPLAYSEQ/2-Data/raw/Data_neurospin/sub-16/run02_raw.fif...
    Read a total of 13 projection items:
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
    Range : 33000 ... 418999 =     33.000 ...   418.999 secs
Ready.
Reading 0 ... 385999  =      0.000 ...   385.999 secs...


/var/folders/qn/h72tf80n225331v43m72lykw0000gn/T/ipykernel_62998/3439713663.py:1: RuntimeWarning: This file contains raw Internal Active Shielding data. It may be distorted. Elekta recommends it be run through MaxFilter to produce reliable results. Consider closing the file and running MaxFilter on the data.
  raw = mne.io.read_raw_fif(data_path, allow_maxshield=True, preload=True)


In [24]:
sub=subject
# Load raw data
raw = mne.io.read_raw_fif(data_path, allow_maxshield=True, preload=True, verbose=False)

# Annotate muscle artifacts
annot_muscle, scores_muscle = mne.preprocessing.annotate_muscle_zscore(
    raw,
    ch_type="mag",
    threshold=5,
    min_length_good=0.2,
    filter_freq=[100, 300],
)

# Set annotations
raw.set_annotations(annot_muscle)

# Remove muscle artifacts
raw_cleaned = raw.copy().filter(l_freq=None, h_freq=40.0)

# Update bad channels from JSON file
raw_cleaned.info['bads'] = bad_channels_dict[f'sub-{subject}'][run_str]

# Extract events and event IDs
if lab == 'icm':
    events = mne.find_events(raw_cleaned, min_duration=0.01, verbose=False)
    events = events[events[:, 2] % 10 != 0, :]
    event_ids = {"Fixation": 9, "Reproduction": 99, "Resting_state": 128}
else:
    events = mne.find_events(raw_cleaned, mask_type="not_and", mask=2**6 + 2**7, verbose=False)
    event_ids = event_dict

# Create BIDS path
bids_path = BIDSPath(
    subject=f'{sub:02}',
    task='reproduction',
    run=run,
    datatype='meg',
    root=path_bids
)

# Save cleaned data in BIDS format
write_raw_bids(
    raw_cleaned,
    bids_path=bids_path,
    events=events,
    event_id=event_ids,
    overwrite=True,
    format='FIF',
    verbose=False,
    allow_preload=True
)

# Save MEG calibration and crosstalk files
if lab == 'icm':
    ct_fname = os.path.join(path_bids, "calibration_files/calibration_icm/ct_sparse.fif")
    cal_fname = os.path.join(path_bids, "calibration_files/calibration_icm/sss_cal_3101_160108.dat")
else:
    ct_fname = os.path.join(path_bids, "calibration_files/calibration_neurospin/ct_sparse.fif")
    cal_fname = os.path.join(path_bids, "calibration_files/calibration_neurospin/sss_cal_3176_20240123_2.dat")

write_meg_calibration(calibration=cal_fname, bids_path=bids_path, verbose=False)
write_meg_crosstalk(fname=ct_fname, bids_path=bids_path, verbose=False)

# Plot power spectral density (PSD) and save the figure
fig = raw_cleaned.compute_psd(picks='mag').plot()
fig.savefig(os.path.join(path_bids, f'sub-{sub:02}_run{run:02}_psd.png'))

print(f"Saving {subject} to {path_bids}")

/var/folders/qn/h72tf80n225331v43m72lykw0000gn/T/ipykernel_62998/1770893561.py:3: RuntimeWarning: This file contains raw Internal Active Shielding data. It may be distorted. Elekta recommends it be run through MaxFilter to produce reliable results. Consider closing the file and running MaxFilter on the data.
  raw = mne.io.read_raw_fif(data_path, allow_maxshield=True, preload=True, verbose=False)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1e+02 - 3e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 100.00
- Lower transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 87.50 Hz)
- Upper passband edge: 300.00 Hz
- Upper transition bandwidth: 75.00 Hz (-6 dB cutoff frequency: 337.50 Hz)
- Filter length: 133 samples (0.133 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.4s


Setting up low-pass filter at 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 1651 samples (1.651 s)

Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 331 samples (0.331 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    1.1s


ValueError: The provided raw data contains annotations, but "event_id" does not contain entries for all annotation descriptions. The following entries are missing: BAD_muscle